In [1]:
!nvidia-smi

Mon Oct 28 22:48:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650       WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8                3W /  N/A|      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
from moabb.datasets import Wang2016
from moabb.paradigms import SSVEP
import mne
import numpy as np
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
c:\Users\machi\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


c:\Users\machi\anaconda3\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [3]:
channels = ["Oz", "O1", "O2"]
events = ["12", "15"]

dataset = Wang2016()
dataset.subject_list = list(range(1, 36))
paradigm = SSVEP(channels=channels, n_classes=2, events=["12", "15"])

In [4]:
X, labels, meta = paradigm.get_data(dataset=dataset)

Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create co

In [5]:
X.shape

(420, 3, 1251)

In [58]:
labels.shape

(420,)

In [9]:
def get_subject_data(full_data, labels, subject):
    num_trials = full_data.shape[0] / 35  # 35 subjects, so num of trials for each subj
    start = int((subject - 1) * num_trials)
    end = int(start + num_trials)
    return full_data[start:end, :, :], labels[start:end]

In [28]:
sub35, labels35 = get_subject_data(X, labels, 1)
sub35.shape

(12, 3, 1251)

In [57]:
labels35.shape

(12,)

In [67]:
def splitData(data, labels, length, fsample, overlap):
    split_data = []
    split_labels = []
    num_samples_split = round(fsample * length)
    step = round(num_samples_split * (1 - overlap))

    for i in range(0, data.shape[0]):
        for j in range(0, data.shape[2], step):
            if (j + num_samples_split) <= data.shape[2]:
                split_data.append(data[i, :, j : j + num_samples_split])
                split_labels.append(labels[i])

    split_data = np.array(split_data)
    split_labels = np.array(split_labels)

    return split_data, split_labels

array([[-1.59641959e+00, -1.29024316e+00,  2.33356927e-01,
         1.96652132e+00,  2.77069624e+00,  2.16668087e+00,
         5.15663026e-01, -1.30728472e+00, -2.39334975e+00,
        -2.27480023e+00, -1.23499874e+00, -1.64923229e-01,
         8.54710804e-02, -5.84910425e-01, -1.38729235e+00,
        -1.29159144e+00,  2.13853579e-02,  1.78374289e+00,
         2.77934320e+00,  2.43673874e+00,  1.29397746e+00,
         3.51736219e-01, -2.16372642e-02, -4.70380839e-01,
        -1.86965437e+00, -4.16596744e+00, -6.08354704e+00,
        -6.11395654e+00, -3.89126015e+00, -5.81772408e-01,
         2.13643852e+00,  3.46930598e+00,  3.89393569e+00,
         4.31630083e+00,  5.01602221e+00,  5.37842835e+00,
         4.51909366e+00,  2.18845600e+00, -8.09961504e-01,
        -3.07817048e+00, -3.61261761e+00, -2.49560720e+00,
        -7.70483484e-01,  3.07763301e-01,  9.51710862e-02,
        -1.02322515e+00, -1.89803371e+00, -1.49152972e+00,
         2.18744167e-01,  2.08950293e+00,  2.65585453e+0